In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split


import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K
from tensorflow.keras import optimizers, losses, Model, utils, callbacks
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from transformers import TFAutoModelForSequenceClassification, TFAutoModel, AutoTokenizer


2024-11-28 19:27:22.904591: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-28 19:27:22.912104: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732789642.920703  357286 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732789642.923288  357286 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-28 19:27:22.932475: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
dataset = fetch_20newsgroups(subset='all')

X = pd.Series(dataset['data'])
y = pd.Series(dataset['target'])
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, stratify=y, random_state=19)
y_train = pd.get_dummies(y_train)
y_valid = pd.get_dummies(y_valid)

In [3]:
seq_len = 512
batch_size = 8
transformer = 'distilroberta-base'
tokenizer = AutoTokenizer.from_pretrained(transformer)
encoder = TFAutoModel.from_pretrained(transformer)

input_ids = L.Input(shape=(seq_len,), dtype=tf.int32, name='input_ids')
input_attention_mask = L.Input(shape=(seq_len,), dtype=tf.int32, name='attention_mask')

embeddings = encoder.roberta([input_ids, input_attention_mask])[0]
pooled = L.GlobalAveragePooling1D()(embeddings)
outputs = L.Dense(20)(pooled)

model = Model(inputs=[input_ids, input_attention_mask], outputs=outputs)
model.compile(optimizer=optimizers.Adam(lr=1e-5), loss=losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

I0000 00:00:1732789731.442516  357286 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1210 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaModel were initialized from the PyTorch model.
If your task is simi

ValueError: Exception encountered when calling layer 'roberta' (type TFRobertaMainLayer).

Data of type <class 'keras.src.backend.common.keras_tensor.KerasTensor'> is not allowed only (<class 'tensorflow.python.framework.tensor.Tensor'>, <class 'bool'>, <class 'int'>, <class 'transformers.utils.generic.ModelOutput'>, <class 'tuple'>, <class 'list'>, <class 'dict'>, <class 'numpy.ndarray'>) is accepted for input_ids.

Call arguments received by layer 'roberta' (type TFRobertaMainLayer):
  • input_ids=['<KerasTensor shape=(None, 512), dtype=int32, sparse=False, name=input_ids>', '<KerasTensor shape=(None, 512), dtype=int32, sparse=False, name=attention_mask>']
  • attention_mask=None
  • token_type_ids=None
  • position_ids=None
  • head_mask=None
  • inputs_embeds=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • past_key_values=None
  • use_cache=None
  • output_attentions=None
  • output_hidden_states=None
  • return_dict=None
  • training=False

In [4]:
tokenized_inputs_train = tokenizer(
    X_train.values.tolist(), max_length=seq_len, truncation=True, padding='max_length', return_tensors='tf'
)
tokenized_inputs_valid = tokenizer(
    X_valid.values.tolist(), max_length=seq_len, truncation=True, padding='max_length', return_tensors='tf'
)

In [5]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(tokenized_inputs_train), list(y_train.values)))
valid_dataset = tf.data.Dataset.from_tensor_slices((dict(tokenized_inputs_valid), list(y_valid.values)))